In [2]:
import os
import subprocess
import time

def setup_environment():
    """设置环境变量，与 Bash 脚本一致"""
    os.environ["LD_LIBRARY_PATH"] = (
        "/home/cc/intel/oneapi/tbb/latest/lib:" + os.environ.get("LD_LIBRARY_PATH", "")
    )
    os.environ["OMP_PROC_BIND"] = "close"
    os.environ["OMP_PLACES"] = "threads"
    os.environ["OMP_DYNAMIC"] = "true"

def create_output_directories(base_output_dir, subdirs):
    """创建输出目录"""
    for subdir in subdirs:
        dir_path = os.path.join(base_output_dir, subdir)
        if not os.path.exists(dir_path):
            print(f"创建目录：{dir_path}")
            os.makedirs(dir_path)

def run_test(algorithm, data_graph, insert_graph, query_graph, output_file, time_limit, threads, auto_tuning, timeout_duration=None):
    """运行单次测试"""
    cmd = [
        "./build/csm",
        "-a", algorithm,
        "-d", data_graph,
        "-u", insert_graph,
        "-q", query_graph,
        "--time-limit", str(time_limit),
        "--report-initial", "off",
        "-t", str(threads)
    ]
    if auto_tuning is not None:
        cmd.extend(["--auto-tuning", str(auto_tuning)])

    # 使用 sudo 执行，输出追加到日志文件
    try:
        print(f"执行测试：{query_graph}")
        with open(output_file, "a") as f:
            process = subprocess.run(
                ["sudo"] + cmd,
                stdout=f,
                stderr=subprocess.PIPE,
                text=True,
                timeout=timeout_duration
            )
        if process.returncode == 0:
            print(f"完成测试：{query_graph}")
        else:
            print(f"测试失败：{query_graph}，返回码：{process.returncode}")
            print(f"错误输出：{process.stderr}")
    except subprocess.TimeoutExpired:
        print(f"测试超时：{query_graph}，超时时间：{timeout_duration}秒")
    except Exception as e:
        print(f"测试出错：{query_graph}，错误：{str(e)}")
    print("-" * 40)

def main():
    # 基础配置
    base_dir = "/home/cc/haibin2"
    dataset = "amazon/7"
    algorithm = "parallel_graphflow"
    time_limit = 180
    output_filename = "parallel_graphflow_Amazon_7_py.txt"

    # 数据图和插入图
    data_graph = os.path.join(base_dir, dataset, "data_graph/data.graph")
    insert_graph = os.path.join(base_dir, dataset, "data_graph/insertion.graph")

    # 输出目录
    base_output_dir = os.path.join(base_dir, "logs_txt/amazon/Parallel_graphflow")
    subdirs = ["sparse_7", "dense_7", "tree_7"]

    # 查询图目录和线程配置
    configs = [
        {
            "query_dir": os.path.join(base_dir, "amazon/7_self/sparse"),
            "output_subdir": "sparse_7",
            "threads": 6,
            "auto_tuning": None,  # 未指定 auto-tuning
            "timeout": None
        },
        {
            "query_dir": os.path.join(base_dir, "amazon/7_self/dense"),
            "output_subdir": "dense_7",
            "threads": 16,
            "auto_tuning": None,
            "timeout": None
        },
        {
            "query_dir": os.path.join(base_dir, "amazon/7_self/tree"),
            "output_subdir": "tree_7",
            "threads": 32,
            "auto_tuning": 0,
            "timeout": 600
        }
    ]

    # 设置环境变量
    setup_environment()

    # 创建输出目录
    create_output_directories(base_output_dir, subdirs)

    # 遍历每个配置
    for config in configs:
        query_dir = config["query_dir"]
        output_subdir = config["output_subdir"]
        threads = config["threads"]
        auto_tuning = config["auto_tuning"]
        timeout = config["timeout"]

        output_file = os.path.join(base_output_dir, output_subdir, output_filename)
        print(f"\n开始处理目录：{output_subdir}，输出到：{output_file}")

        # 遍历 Q_1 到 Q_99
        for i in range(1, 20):
            query_graph = os.path.join(query_dir, f"Q_{i}")
            if not os.path.exists(query_graph):
                print(f"查询图 {query_graph} 不存在，跳过...")
                continue

            run_test(
                algorithm=algorithm,
                data_graph=data_graph,
                insert_graph=insert_graph,
                query_graph=query_graph,
                output_file=output_file,
                time_limit=time_limit,
                threads=threads,
                auto_tuning=auto_tuning,
                timeout_duration=timeout
            )

        print(f"完成目录：{output_subdir}\n")

if __name__ == "__main__":
    main()


开始处理目录：sparse_7，输出到：/home/cc/haibin2/logs_txt/amazon/Parallel_graphflow/sparse_7/parallel_graphflow_Amazon_7_py.txt
执行测试：/home/cc/haibin2/amazon/7_self/sparse/Q_1
测试失败：/home/cc/haibin2/amazon/7_self/sparse/Q_1，返回码：255
错误输出：
----------------------------------------
查询图 /home/cc/haibin2/amazon/7_self/sparse/Q_2 不存在，跳过...
查询图 /home/cc/haibin2/amazon/7_self/sparse/Q_3 不存在，跳过...
执行测试：/home/cc/haibin2/amazon/7_self/sparse/Q_4
测试失败：/home/cc/haibin2/amazon/7_self/sparse/Q_4，返回码：255
错误输出：
----------------------------------------
查询图 /home/cc/haibin2/amazon/7_self/sparse/Q_5 不存在，跳过...
执行测试：/home/cc/haibin2/amazon/7_self/sparse/Q_6
测试失败：/home/cc/haibin2/amazon/7_self/sparse/Q_6，返回码：255
错误输出：
----------------------------------------
查询图 /home/cc/haibin2/amazon/7_self/sparse/Q_7 不存在，跳过...
执行测试：/home/cc/haibin2/amazon/7_self/sparse/Q_8
测试失败：/home/cc/haibin2/amazon/7_self/sparse/Q_8，返回码：255
错误输出：
----------------------------------------
查询图 /home/cc/haibin2/amazon/7_self/sparse/Q_9 不存在，跳过...
查询图 /ho